In [43]:
from binance.client import Client
import pandas as pd
import numpy as np
import math
import time
from config import api_key, api_secret
from coin_data import data

In [44]:
client = Client(api_key=api_key, api_secret= api_secret)
ticker = 'MANAUSDT'

# 1. Extraer datos de la moneda

- Traemos toda la info de todas las monedas de futuros de binance

In [45]:
info = client.futures_exchange_info()

- Buscamos la moneda que necesitamos
- Y extraemos la información que creamos necesaria
- Esto lo deberíamos ejecutar 1 sola vez para todas las monedas y guardarlo para solo consultarlo como diccionario y no extraer la información cada vez
- Aunque los datos traen la precisión para el precio, en algunas monedas está mal. Por lo tanto, se crea la función que retorna la cantidad de decimales a través de otra propiedad listada

In [46]:
def get_precision(texto:str) -> int:
    for i in range(2, len(texto)):
        if texto[i] == '1':
            return i-1

In [47]:
for d in info['symbols']:
    if d['pair']==ticker:
        print(d)
        print("")
        print(f"Moneda: {d['pair']}\nSubtipo: {d['underlyingSubType']}\nPrecisión Precio: {get_precision(d['filters'][0]['tickSize'])} decimales")


{'symbol': 'MANAUSDT', 'pair': 'MANAUSDT', 'contractType': 'PERPETUAL', 'deliveryDate': 4133404800000, 'onboardDate': 1615705200000, 'status': 'TRADING', 'maintMarginPercent': '2.5000', 'requiredMarginPercent': '5.0000', 'baseAsset': 'MANA', 'quoteAsset': 'USDT', 'marginAsset': 'USDT', 'pricePrecision': 4, 'quantityPrecision': 0, 'baseAssetPrecision': 8, 'quotePrecision': 8, 'underlyingType': 'COIN', 'underlyingSubType': ['Metaverse'], 'settlePlan': 0, 'triggerProtect': '0.0500', 'liquidationFee': '0.010000', 'marketTakeBound': '0.10', 'filters': [{'minPrice': '0.0136', 'maxPrice': '100000', 'filterType': 'PRICE_FILTER', 'tickSize': '0.0001'}, {'stepSize': '1', 'filterType': 'LOT_SIZE', 'maxQty': '1000000', 'minQty': '1'}, {'stepSize': '1', 'filterType': 'MARKET_LOT_SIZE', 'maxQty': '100000', 'minQty': '1'}, {'limit': 200, 'filterType': 'MAX_NUM_ORDERS'}, {'limit': 10, 'filterType': 'MAX_NUM_ALGO_ORDERS'}, {'notional': '5', 'filterType': 'MIN_NOTIONAL'}, {'multiplierDown': '0.9000', 'm

# 2. Obtener Shock Points

# a. Libro de ordenes

- Vamos a obtener el libro de órdenes de la moneda
- Luego separamos las ventas de las compras y las colocamos en un dataframe para mejor organización
- Como nota, lo que trae el libro no es la cantidad total de monedas por bloque sino el total de USDT
- En el caso de las ventas, para mostrarlo como lo vemos en Binance (mayor a menor) ordenamis descebdentemente

In [48]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
#order_book

In [49]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)
ventas

/tmp/ipykernel_893/4237633481.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)


,precio,total_usdt
999,1.1531,13
998,1.1530,552
997,1.1525,4950
996,1.1524,15
995,1.1522,81
...,...,...
4,1.0200,6511
3,1.0199,6692
2,1.0198,4046
1,1.0197,3549


In [50]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','total_usdt'])
compras

,precio,total_usdt
0,1.0195,6
1,1.0194,424
2,1.0193,695
3,1.0192,5119
4,1.0191,7481
...,...,...
995,0.9070,1189
996,0.9069,141
997,0.9068,7
998,0.9067,38


## b. Obtención de rangos de precios

- Para obtener los rangos y obtener los puntos a cómo lo hace el gafas, debemos crear rangos
- Para crear los rangos necesitamos el precio mínimo y máximo para redondearlos hacia abajo y arriba respecto a los múltiplos de los libros

### i. Obtención de máximos y mínimos precios

- Para poder hacer esto, tenemos que decirle al Dataframe qué tipo de valores son las columnas
- Esto es porque, como se puede observar a continuación, los marca como object

In [51]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   precio      1000 non-null   object
 1   total_usdt  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


- Para modificar el tipo de objetos de las columnas, hacemos:

In [52]:
ventas[['precio','total_usdt']] = ventas[['precio','total_usdt']].astype('float')
ventas

,precio,total_usdt
999,1.1531,13.0
998,1.1530,552.0
997,1.1525,4950.0
996,1.1524,15.0
995,1.1522,81.0
...,...,...
4,1.0200,6511.0
3,1.0199,6692.0
2,1.0198,4046.0
1,1.0197,3549.0


In [53]:
compras[['precio','total_usdt']] = compras[['precio','total_usdt']].astype('float')
compras

,precio,total_usdt
0,1.0195,6.0
1,1.0194,424.0
2,1.0193,695.0
3,1.0192,5119.0
4,1.0191,7481.0
...,...,...
995,0.9070,1189.0
996,0.9069,141.0
997,0.9068,7.0
998,0.9067,38.0


- Ahora sí, obtenemos los máximos y mínimos

In [54]:
max_ventas = ventas['precio'].max()
min_ventas = ventas['precio'].min()
print('Ventas(mín-máx): {} - {}'.format(min_ventas, max_ventas))

max_compras = compras['precio'].max()
min_compras = compras['precio'].min()
print('Compras(mín-máx): {} - {}'.format(min_compras, max_compras))

Ventas(mín-máx): 1.0196 - 1.1531
Compras(mín-máx): 0.9066 - 1.0195


### ii. Obtención de Dataframes con los Rangos

- Para esto, necesitamos tener los múltiplos de las moendas cómo lo hace el gafas
- Desafortunadamante, aún no encuentro cómo obtener los múltiplos de binance por lo que toca escribirlos en el archivo coin_data.py
- Para obtener el inicio de inicio y final de un rango, hacemos:

1. (precio_minimo / multiplo) ; donde el múltiplo es el que obtenemos de coind_data en i1
2. Luego redondeamos al entero inferior con math.floor()
3. Y multiplicamos por el múltiplo nuevamente para obtener el inicio del rango
4. (precio_maximo / multiplo)
5. redondeamos hacia el entero superior con math.ceil()
6. Y multiplicamos por el múltiplo nuevamente para obtener el máximo del rango

#### Obtención de valores iniciales y finales de rango con primer múltiplo

In [55]:
# Ventas
inicio_rango_ventas_1 = ( math.floor( (min_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_ventas_1 = ( math.ceil( (max_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_1,final_rango_ventas_1))
# Compras
inicio_rango_compras_1 = ( math.floor( (min_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_compras_1 = ( math.ceil( (max_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_1,final_rango_compras_1))

Rango Ventas: 1.01 - 1.16
Rango Compras: 0.9 - 1.02


#### Obtención de valores iniciales y finales de rango con segundo múltiplo

In [56]:
# Ventas
inicio_rango_ventas_2 = ( math.floor( (min_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_ventas_2 = ( math.ceil( (max_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_2,final_rango_ventas_2))
# Compras
inicio_rango_compras_2 = ( math.floor( (min_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_compras_2 = ( math.ceil( (max_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_2,final_rango_compras_2))

Rango Ventas: 1.0190000000000001 - 1.154
Rango Compras: 0.906 - 1.02


### iii. Obtención de los dataframes agrupados por rangos

- Ya que tenemos los valores inicial y final de los rangos, creamos el Dataframe con datos agrupados por rangos
- Solo que, antes tenemos que tener un rango con incrementos de acuerdo al múltiplo. Para esto usamos np.arange()

- Con múltiplo 1

In [57]:
rango_v_1 = np.arange(inicio_rango_ventas_1, final_rango_ventas_1+data[ticker]['i1'], data[ticker]['i1'])
ventas_ag_1 = ventas.groupby(pd.cut(ventas.precio,rango_v_1)).sum()
print('Ventas agrupadas segun incremento 1')
print(rango_v_1)
print(ventas_ag_1.sort_index(ascending=False))
rango_c_1 = np.arange(inicio_rango_compras_1, final_rango_compras_1+data[ticker]['i1'], data[ticker]['i1'])
compras_ag_1 = compras.groupby(pd.cut(compras.precio,rango_c_1)).sum()
print('\nCompras agrupadas segun incremento 1')
print(rango_c_1)
print(compras_ag_1.sort_index())

Ventas agrupadas segun incremento 1
[1.01 1.02 1.03 1.04 1.05 1.06 1.07 1.08 1.09 1.1  1.11 1.12 1.13 1.14
 1.15 1.16]
                precio  total_usdt
precio                            
(1.15, 1.16]   12.6708     16908.0
(1.14, 1.15]   67.5587     34807.0
(1.13, 1.14]   64.7038    287442.0
(1.12, 1.13]   70.8710    112394.0
(1.11, 1.12]   68.0162     90023.0
(1.1, 1.11]    75.1496    120003.0
(1.09, 1.1]    75.5578    116536.0
(1.08, 1.09]   84.6319     54391.0
(1.07, 1.08]   82.7861    114619.0
(1.06, 1.07]   88.4056    132931.0
(1.05, 1.06]   87.5672    243135.0
(1.04, 1.05]   94.0434    386791.0
(1.03, 1.04]   99.3564    616840.0
(1.02, 1.03]  102.5050    963217.0
(1.01, 1.02]    5.0990     21039.0

Compras agrupadas segun incremento 1
[0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 1.   1.01 1.02]
               precio  total_usdt
precio                           
(0.9, 0.91]   25.4344     60217.0
(0.91, 0.92]  75.0256     79787.0
(0.92, 0.93]  79.5494    234898.0
(0.93, 0.94

- con múltiplo 2

In [58]:
rango_v_2 = np.arange(inicio_rango_ventas_2, final_rango_ventas_2+data[ticker]['i2'], data[ticker]['i2'])
ventas_ag_2 = ventas.groupby(pd.cut(ventas.precio,rango_v_2)).sum()
print('Ventas agrupadas segun incremento 2')
#print(rango_v_2)
print(ventas_ag_2.sort_index(ascending=False))
rango_c_2 = np.arange(inicio_rango_compras_2, final_rango_compras_2+data[ticker]['i2'], data[ticker]['i2'])
compras_ag_2 = compras.groupby(pd.cut(compras.precio,rango_c_2)).sum()
print('\nCompras agrupadas segun incremento 2')
#print(rango_c_2)
print(compras_ag_2.sort_index(ascending=False))

Ventas agrupadas segun incremento 2
                 precio  total_usdt
precio                             
(1.153, 1.154]   2.3061       565.0
(1.152, 1.153]   5.7612      5244.0
(1.151, 1.152]   2.3025     10933.0
(1.15, 1.151]    3.4510      1713.0
(1.149, 1.15]    6.8967       367.0
...                 ...         ...
(1.023, 1.024]  10.2345    100627.0
(1.022, 1.023]  10.2245     73742.0
(1.021, 1.022]   9.1935     59904.0
(1.02, 1.021]   10.2055     45756.0
(1.019, 1.02]    5.0990     21039.0

[135 rows x 2 columns]

Compras agrupadas segun incremento 2
                 precio  total_usdt
precio                             
(1.019, 1.02]    5.0965     13725.0
(1.018, 1.019]  10.1855    108597.0
(1.017, 1.018]  10.1755     45941.0
(1.016, 1.017]  10.1655    120564.0
(1.015, 1.016]  10.1555     62608.0
...                 ...         ...
(0.91, 0.911]    9.1055      2008.0
(0.909, 0.91]    9.0955     45078.0
(0.908, 0.909]   5.4515      3401.0
(0.907, 0.908]   6.3534      9797.0
(0

#### Obtención de los shock points

- Ahora, para obtener los shock point, necesitamos obtener el dato donde el total usdt es máximo
- Traer el índice y extraer:
    - Para ventas: el lado derecho del intervalo. Porque es la parte superior del intervalo
    - Para compras: el lado izquiero del intervalo. Porque es la parte inferior del intervalo

In [63]:
from numpy import sort


sp_v_1 = ventas_ag_1['total_usdt'].idxmax().right
#print(sp_v_1)
#print(ventas_ag_2)
#print(ventas_ag_2['total_usdt'].max())
#print(ventas_ag_2['total_usdt'].idxmax())
#print(ventas_ag_2['total_usdt'].idxmax())
sp_v_2 = ventas_ag_2['total_usdt'].idxmax().right
sp_v = []
sp_v.append(sp_v_1)
sp_v.append(sp_v_2)
sp_v = sort(sp_v)
sp_v_1 = sp_v[1]
sp_v_2 = sp_v[0]
print(f'Ventas:\n***************\n2.  {sp_v_1}\n1.  {sp_v_2}')

Ventas:
***************
2.  1.03
1.  1.029


In [60]:
sp_c_1 = compras_ag_1['total_usdt'].idxmax().left
#print(sp_c_1)
#print(compras_ag_2)
#print(compras_ag_2['total_usdt'].max())
#print(compras_ag_2['total_usdt'].idxmax())
sp_c_2 = compras_ag_2['total_usdt'].idxmax().left
#print(sp_c_2)
sp_c = []
sp_c.append(sp_c_1)
sp_c.append(sp_c_2)
sp_c = sort(sp_c)
sp_c_1 = sp_c[1]
sp_c_2 = sp_c[0]
print(f'Compras:\n***************\n1.  {sp_c_1}\n2.  {sp_c_2}')

Compras:
***************
1.  1.01
2.  0.989


# 3. Analizar Shock Points

- El análisis de los puntos se hará de la siguiente manera:
    - Para ventas, obtenemos la distancia o porcentaje entre puntos de venta: pct_v = ( (venta_fin - venta_inicio)/venta_din ) * 100
    - Para compras, lo calculasmo así: pct_c = ( (compra_inicio - compra_fin)/compra_inicio ) * 100

In [64]:
pct_ventas = round( ( (sp_v_1 - sp_v_2) / sp_v_2 ) * 100, 2 )
print(f'Distancia entre ventas: {pct_ventas}%')

pct_compras = round( ( (sp_c_1 - sp_c_2) / sp_c_1 ) * 100, 2)

Distancia entre ventas: 0.1%


# 4. Resultado del Análisis de los Shock Points

# 5. Calcular SL y TP

# 6. Ejecutar órdenes